In [0]:
import sys
print(sys.version)

CURR_DIR = '/content/drive/My Drive/google_colab_work/detailed_probabilistic_robotics/'
sys.path.append(CURR_DIR)
sys.path.append(CURR_DIR + 'scripts/')

3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]


In [0]:
from mcl import *
from kf import *

3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
class EstimatedLandmark(Landmark):
    def __init__(self):
        super().__init__(x=0, y=0)
        self.cov = None

    def draw(self, ax, elems):
        if self.cov is None:
            return
        
        star = ax.scatter(self.pos[0], self.pos[1], s=100, marker='*', label='landmarks', color='blue')
        elems.append(star)
        elems.append(ax.text(self.pos[0], self.pos[1], 'id:'+str(self.id), fontsize=10))

        ellipse = sigma_ellipse(p=self.pos, cov=self.cov, n=3)
        elems.append(ax.add_patch(ellipse))

In [0]:
class MapParticle(Particle):
    def __init__(self, init_pose, weight, landmark_num):
        super().__init__(init_pose, weight)
        self.map = Map()
        for i in range(landmark_num):
            self.map.append_landmark(EstimatedLandmark())

    def observation_update(self, obs, distance_std_rate, direction_std):
        for z, obs_id in obs:
            landmark = self.map.landmarks[obs_id]
            if landmark.cov is None:
                self.init_landmark_estimation(landmark, z, distance_std_rate, direction_std)
            else:
                self.observation_update_landmark(landmark, z, distance_std_rate, direction_std)
        
    def init_landmark_estimation(self, landmark, z, distance_std_rate, direction_std):
        landmark.pos = z[0]*np.array([math.cos(self.pose[2]+z[1]), math.sin(self.pose[2]+z[1])]) + self.pose[0:2]
        H = -matH(self.pose, landmark.pos)[0:2, 0:2]
        Q = matQ(distance_std_rate*z[0], direction_std)
        landmark.cov = np.linalg.inv(H.T.dot(np.linalg.inv(Q)).dot(H))

    def observation_update_landmark(self, landmark, z, distance_std_rate, direction_std):
        landmark_before = copy.deepcopy(landmark)
        estimated_z = IdealCamera.observation_function(self.pose, landmark_before.pos)
        if estimated_z[0] < 0.01: ### If it's too close, then return to avoid miscalculating
            return

        H = -matH(self.pose, landmark_before.pos)[0:2, 0:2]
        Q = matQ(distance_std_rate*estimated_z[0], direction_std)
        K = landmark_before.cov.dot(H.T).dot(np.linalg.inv(Q + H.dot(landmark_before.cov).dot(H.T)))

        Q_z = H.dot(landmark_before.cov).dot(H.T) + Q
        particle_pose_likelihood_dist = scipy.stats.multivariate_normal(mean=estimated_z, cov=Q_z)
        self.weight *= particle_pose_likelihood_dist.pdf(z)

        landmark.pos = K.dot(z - estimated_z) + landmark_before.pos ### Update landmark pos
        landmark.cov = (np.eye(2) - K.dot(H)).dot(landmark_before.cov) ### Update landmark cov

In [0]:
class FastSLAM(Mcl):
    def __init__(
        self, init_pose, particle_num, landmark_num,
        motion_noise_stds={'nn':0.19, 'no':0.001, 'on':0.13, 'oo':0.20},
        distance_std_rate=0.14, direction_std=0.05
    ):
        super().__init__(None, init_pose, particle_num, motion_noise_stds, distance_std_rate, direction_std)
        self.particles = [MapParticle(init_pose, 1.0/particle_num, landmark_num) for i in range(particle_num)]
        self.max_likeli = self.particles[0]

    def observation_update(self, obs):
        for particle in self.particles:
            particle.observation_update(obs, self.distance_std_rate, self.direction_std)
        self.set_max_likeli()
        self.resampling()

    def draw(self, ax, elems):
        super().draw(ax, elems)
        self.max_likeli.map.draw(ax, elems)

In [0]:
def trial():
    time_span = 30
    time_interval = 0.1
    world = World(time_span, time_interval, debug=False)

    m = Map()
    for landmark in [(-4,2), (2,-3), (3,3)]:
        m.append_landmark(Landmark(*landmark))
    world.append(m)

    init_pose = np.array([0, 0, math.radians(0.0)])
    particle_num = 100
    landmark_num = len(m.landmarks)
    fastslam_estimator = FastSLAM(init_pose, particle_num, landmark_num)

    nu = 0.2
    omega = math.radians(10.0)
    circling_agent = EstimationAgent(nu, omega, time_interval, estimator=fastslam_estimator)

    robot = Robot(init_pose, agent=circling_agent, sensor=Camera(m), color='red')
    world.append(robot)

    world.draw()
    return world

In [0]:
world = trial()
world.ani

Output hidden; open in https://colab.research.google.com to view.